In [ ]:
import requests
import smtplib
from email.mime.text import MIMEText
from datetime import datetime
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()


# NOAA Station ID
STATION_ID = "8534720"

# NOAA API endpoints
TIDE_URL = (
    f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?"
    f"product=predictions&application=chatgpt-tide-app&begin_date=latest&"
    f"end_date=latest&datum=MLLW&station={STATION_ID}&time_zone=lst_ldt&"
    f"units=english&interval=hilo&format=json"
)

TEMP_URL = (
    f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?"
    f"product=water_temperature&application=chatgpt-tide-app&station={STATION_ID}&"
    f"date=latest&units=english&time_zone=lst_ldt&format=json"
)

# Email config
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_SENDER = "nbenelli.waterreport@gmail.com"
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
EMAIL_RECIPIENT = "nick.benelli12@gmail.com"


def fetch_tide_data(station_id="8534720", date=None):
    if date is None:
        date = datetime.now().strftime("%Y%m%d")

    # tide_url = (
    #     f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?"
    #     f"product=predictions&application=chatgpt-tide-app&begin_date={today}&"
    #     f"end_date={today}&datum=MLLW&station={STATION_ID}&time_zone=lst_ldt&"
    #     f"units=english&interval=hilo&format=json"
    # )
    base_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

    params = {
        "product": "predictions",
        "application": "chatgpt-tide-app",
        "begin_date": date,
        "end_date": date,
        "datum": "MLLW",
        "station": station_id,
        "time_zone": "lst_ldt",
        "units": "english",
        "interval": "hilo",
        "format": "json",
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()

    return data["predictions"]


def fetch_water_temp(station_id: str = "8534720"):
    # response = requests.get(TEMP_URL)
    base_url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"

    params = {
        "product": "water_temperature",
        "application": "chatgpt-tide-app",
        "station": station_id,
        "date": "latest",
        "units": "english",
        "time_zone": "lst_ldt",
        "format": "json",
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    return data["data"][0]["v"] + " °F"


def send_email(subject, body):
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = EMAIL_SENDER
    msg["To"] = EMAIL_RECIPIENT

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.send_message(msg)


water_temp = fetch_water_temp()


display(pd.DataFrame(fetch_tide_data()))

,t,v,type
0,2025-07-03 01:57,3.589,H
1,2025-07-03 08:23,0.713,L
2,2025-07-03 14:47,3.894,H
3,2025-07-03 21:02,1.147,L


In [27]:
tide_data = fetch_tide_data()

In [28]:
from datetime import datetime, time


def filter_daytime_tides(tides, start_hour=7, end_hour=19):
    """Return only tides that occur between start_hour and end_hour (24-hour format)."""
    result = []
    for tide in tides:
        tide_time = datetime.strptime(tide["t"], "%Y-%m-%d %H:%M")
        if time(start_hour) <= tide_time.time() <= time(end_hour):
            result.append(tide)
    return result

In [29]:
pd.DataFrame(filter_daytime_tides(tide_data))

day_tide_data = filter_daytime_tides(tide_data)

In [30]:
from datetime import datetime


def format_tide_for_email(tide_events):
    formatted = []
    for tide in tide_events:
        dt = datetime.strptime(tide["t"], "%Y-%m-%d %H:%M")
        time_str = dt.strftime("%-I:%M %p")  # e.g., 2:47 PM
        tide_type = "High Tide" if tide["type"] == "H" else "Low Tide"
        height = float(tide["v"])
        formatted.append(f"{tide_type} at {time_str} — {height:.1f} ft")
    return "\n".join(formatted)


print(format_tide_for_email(day_tide_data))

Low Tide at 8:23 AM — 0.7 ft
High Tide at 2:47 PM — 3.9 ft


In [31]:
from datetime import datetime


def format_tide_for_email(tide_events):
    formatted = []
    for tide in tide_events:
        dt = datetime.strptime(tide["t"], "%Y-%m-%d %H:%M")
        time_str = dt.strftime("%-I:%M %p")  # e.g., 2:47 PM
        tide_type = "High Tide" if tide["type"] == "H" else "Low Tide"
        height = float(tide["v"])
        formatted.append(f"{tide_type} at {time_str} — {height:.1f} ft")
    return "\n".join(formatted)


def generate_email_body(tide_events, water_temp_fahrenheit):
    today = datetime.now().strftime("%A, %B %d, %Y")

    tide_section = format_tide_for_email(tide_events)

    body = (
        f"🌊 Daily Water Report – {today} 🌊\n\n"
        f"🌡️ Water Temperature: {water_temp_fahrenheit}\n\n"
        f"🌊 Tides:\n{tide_section}\n\n"
        # f"Have a great day on the water!"
    )
    return body

In [32]:
# Example input data
# tide_data = [
#     {'t': '2025-07-03 08:23', 'v': '0.713', 'type': 'L'},
#     {'t': '2025-07-03 14:47', 'v': '3.894', 'type': 'H'}
# ]
# water_temp = "72.5 °F"

email_body = generate_email_body(day_tide_data, water_temp)
print(email_body)

🌊 Daily Water Report – Thursday, July 03, 2025 🌊

🌡️ Water Temperature: 70.9 °F

🌊 Tides:
Low Tide at 8:23 AM — 0.7 ft
High Tide at 2:47 PM — 3.9 ft




In [33]:
send_email(subject="Daily Water Report", body=email_body)

In [1]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")

'2025-07-03'

In [3]:
lines = ['a', 'b', 'c']
print('\n'.join(lines) + "\n\n")
'\n'.join(lines) + "\n\n"

a
b
c




'a\nb\nc\n\n'